In [1]:
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, re
import time

In [3]:
def getFileName(name, date=None, stimDataFile=None, lickDataFile=None):
    dir_path = os.path.join("../../data/cage1",name)
     
    if stimDataFile:
        stimDataFile = [stimDataFile]
        actualStimDataFile = None
    elif date:
        stimDataFile = [os.path.join(dir_path, item) for item in os.listdir(dir_path) if date in item and 
                        os.path.splitext(item)[1] == ".data" and
                        re.match("^\d",item)]
        actualStimDataFile = [os.path.join(dir_path, item) for item in os.listdir(dir_path) if date in item and 
                        os.path.splitext(item)[1] == ".data" and
                        re.match("^a",item)]
    else:
        stimDataFile = None
        actualStimDataFile = None
        
    if lickDataFile:
        lickDataFile = [lickDataFile]
    elif date:
        lickDataFile = [os.path.join(dir_path, item) for item in os.listdir(dir_path) if '.'.join([date[0:2],date[2:4],date[4:6]]) in item and 
                        os.path.splitext(item)[1] == ".csv" and
                        re.match("^Kaleidoscope",item)]
    else:
        lickDataFile = None
    
    return stimDataFile, actualStimDataFile, lickDataFile

In [2]:
def show_session_raw(stim_source, lick_source, start_time, output=None):
    def convertExcelDateToTimestamp(excel_T):
        """Convert Excel datetime format into Unix timestamp."""
        return (excel_T - 25569)*86400 - 8*3600

    lick_ticks = np.array([i-start_time for i in list(map(
                                    convertExcelDateToTimestamp,
                                    lick_source))])

    title = stim_source['session_name']
    plt.figure(figsize=(240,5))
    tick = 0
    for each_trial in stim_source['trial_sequence']:
        for color_name, lasting in each_trial['color_sequence']:
            color_val = 0 if color_name == "black" else 1
            plt.plot([tick, tick, tick+lasting, tick+lasting], [0, color_val, color_val, 0], 
                     color=color_name, 
                     fillstyle="full")
            tick += lasting

    for each_lick in lick_ticks[(lick_ticks>=0)&(lick_ticks<=tick)]:
        plt.plot([each_lick, each_lick], [0, 2], color="red", alpha=0.5)

    plt.xlim((0, tick))
    plt.ylim((0,2))
    if output:
        plt.savefig(output,bbox_inches="tight")
        print("saved chart:", output)
    else:
        plt.show()

In [6]:
demo = "../../data/cage1/Auden"
mouselist = ['Auden', 'Bartolo', 'Charles', 'Dorian', 'Elliott', 'Frodo']
today = "180320"

for each_mouse in mouselist:
    stimDataFile, _, lickDataFile = getFileName(each_mouse, today)
    if len(stimDataFile) == 0:
        continue
    
    with open(stimDataFile[0], 'r') as inputfile:
        false = False
        true = True
        demo_stim = eval(inputfile.readline())
        start_time = eval(inputfile.readline())
    demo_lick = pd.read_csv(lickDataFile[0])
    lick_filter = demo_lick[demo_lick.SystemMsg == "lick"]['DateTime'].values
    
    show_session_raw(demo_stim[0], 
                 lick_filter, 
                 start_time, 
                 "chart/raw/{name}_{date}.png".format(name=each_mouse, date=today))
        

saved chart: chart/Auden_180320.png
saved chart: chart/Bartolo_180320.png
saved chart: chart/Charles_180320.png
saved chart: chart/Dorian_180320.png
saved chart: chart/Elliott_180320.png
saved chart: chart/Frodo_180320.png
